In [2]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import scipy as sp
import csv

In [3]:
movies_csv = 'C:/Users/Molina/Documents/Untitled Folder/movie recommender/movies.csv'
ratings_csv = 'C:/Users/Molina/Documents/Untitled Folder/movie recommender/ratings.csv'

In [4]:
movies_df = pd.read_csv(movies_csv)
ratings_df = pd.read_csv(ratings_csv)

In [5]:
movies_df

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


In [5]:
ratings_df

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [6]:
#merge the two dataframes through the movie id
rated_movies = ratings_df.merge(movies_df, left_on = 'movieId', right_on = 'movieId', suffixes = ['_user', ''])
rated_movies

,userId,movieId,rating,timestamp,title,genres
0,1,1,4.0,964982703,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,5,1,4.0,847434962,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
2,7,1,4.5,1106635946,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
3,15,1,2.5,1510577970,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
4,17,1,4.5,1305696483,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
...,...,...,...,...,...,...
100831,610,160341,2.5,1479545749,Bloodmoon (1997),Action|Thriller
100832,610,160527,4.5,1479544998,Sympathy for the Underdog (1971),Action|Crime|Drama
100833,610,160836,3.0,1493844794,Hazard (2005),Action|Drama|Thriller
100834,610,163937,3.5,1493848789,Blair Witch (2016),Horror|Thriller


In [7]:
pivot_table = rated_movies.pivot_table(index = ['userId'], columns = ['title'], values = 'rating')
pivot_table = pivot_table.apply(lambda x: (x - np.mean(x))/(np.max(x) - np.min(x)), axis = 1)
pivot_table.fillna(0, inplace=True)
pivot_table = pivot_table.T
pivot_table = pivot_table.loc[:, (pivot_table != 0).any(axis=0)]
pivot_table

userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.069210
'Hellboy': The Seeds of Creation (2004),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
'Round Midnight (1986),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
'Salem's Lot (2004),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
'Til There Was You (1997),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.373012,0.0,0.0,0.0,0.0,0.303517,0.0,0.000000
xXx (2002),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,-0.565217,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.081294,0.0,-0.375235
xXx: State of the Union (2005),0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,-0.486346


In [10]:
piv_sparse = sp.sparse.csr_matrix(pivot_table.values)
piv_sparse

<9719x609 sparse matrix of type '<class 'numpy.float64'>'
	with 100592 stored elements in Compressed Sparse Row format>

In [12]:
movie_similarity = cosine_similarity(piv_sparse)
movie_sim_df = pd.DataFrame(movie_similarity, index = pivot_table.index, columns = pivot_table.index)
movie_sim_df

title,'71 (2014),'Hellboy': The Seeds of Creation (2004),'Round Midnight (1986),'Salem's Lot (2004),'Til There Was You (1997),'Tis the Season for Love (2015),"'burbs, The (1989)",'night Mother (1986),(500) Days of Summer (2009),*batteries not included (1987),...,Zulu (2013),[REC] (2007),[REC]² (2009),[REC]³ 3 Génesis (2012),anohana: The Flower We Saw That Day - The Movie (2013),eXistenZ (1999),xXx (2002),xXx: State of the Union (2005),¡Three Amigos! (1986),À nous la liberté (Freedom for Us) (1931)
title,,,,,,,,,,,,,,,,,,,,,
'71 (2014),1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.028512,0.000000,...,0.000000,0.107629,-0.191027,-0.715017,0.0,0.000000,-0.294099,-0.618923,0.000000,0.0
'Hellboy': The Seeds of Creation (2004),0.000000,1.000000,-0.261451,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Round Midnight (1986),0.000000,-0.261451,1.000000,0.000000,0.000000,0.0,0.082930,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Salem's Lot (2004),0.000000,0.000000,0.000000,1.000000,0.938318,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
'Til There Was You (1997),0.000000,0.000000,0.000000,0.938318,1.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
eXistenZ (1999),0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.022397,-0.126390,-0.100798,-0.018863,...,0.000000,0.000000,0.000000,0.000000,0.0,1.000000,0.086119,0.000000,-0.125200,0.0
xXx (2002),-0.294099,0.000000,0.000000,0.000000,0.000000,0.0,0.057446,0.000000,-0.003271,0.088596,...,0.321760,-0.299856,0.136158,0.292262,0.0,0.086119,1.000000,0.462957,-0.009824,0.0
xXx: State of the Union (2005),-0.618923,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,-0.038505,0.000000,...,0.000000,-0.382477,0.118231,0.442541,0.0,0.000000,0.462957,1.000000,0.000000,0.0


In [19]:
def recommendation(title):
    number = 1
    print('Recommended beacause you watched {}:\n'.format(title))
    for movie in movie_sim_df.sort_values(by = title, ascending = False).index[1:6]:
        print(f'#{number}: {movie}, {round(movie_sim_df[movie][title]*100,2)}% match')
        number = number + 1

In [20]:
recommendation('Toy Story (1995)')

Recommended beacause you watched Toy Story (1995):

#1: Toy Story 2 (1999), 35.17% match
#2: Aladdin (1992), 29.7% match
#3: Toy Story 3 (2010), 27.74% match
#4: Wallace & Gromit: The Wrong Trousers (1993), 25.66% match
#5: Lion King, The (1994), 23.07% match
